# Week 4: Milestone 1 – Smart Tracking System Blockchain Ledger (Draft)

Follow these steps to verify Python↔Ganache↔Smart Contract connection.

1. Setup and Connection

1. **Open Jupyter Notebook**  
2. **Verify Ganache** is running (Desktop App “Quickstart Ethereum” or `ganache-cli`).  
3. **Check RPC port** (default is `7545` in Ganache Desktop).

Below we’ll connect Python → Ganache → our IoTDataStorage contract.

In [28]:
# Cell 1: imports & connection
from web3 import Web3
import json, os

# 1️⃣ Point to your Ganache RPC (update port if needed)
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 2️⃣ Test connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


2. Load Contract ABI and Address

🔗 Now we load the **ABI** exported (in `/artifacts/IoTDataStorageABI.json`) and the **deployed address** from Remix.

In [29]:
# Cell 2: load ABI + contract
abi_path = os.path.join("..", "artifacts", "IoTDataStorageABI.json")
with open(abi_path) as f:
    abi = json.load(f)

# Replace with actual deployed address
contract_address = "0xF4B97C9C61c2EcEb30780c5750EB95694A703B88"

contract = web3.eth.contract(address=contract_address, abi=abi)

# Use the Ganache account that has a balance
web3.eth.default_account = web3.eth.accounts[1]

print(f"▶️ Using account {web3.eth.default_account}")
print(f"✅ Loaded contract at {contract_address}")

▶️ Using account 0x52F44946840D06E788D2785B3FfEbDf36AacA355
✅ Loaded contract at 0xF4B97C9C61c2EcEb30780c5750EB95694A703B88


3. Read-Only Call: getTotalRecords()

Let’s check that our contract is responding:

In [34]:
# Cell 3: read-only call
total = contract.functions.getTotalRecords().call()
print(f"🔢 Total stored entries on‐chain: {total}")

🔢 Total stored entries on‐chain: 1


4. Write a dummy entry

Store one dummy IoT record (TEST001) to prove transactions work.

In [31]:
# Cell 4: store dummy entry
tx = contract.functions.storeData(
    "TEST001",      # dummy ID
    "Temperature", 
    "22.5°C"
).transact({
    "from": web3.eth.default_account,
    "gas": 200_000,
    "gasPrice": web3.to_wei("1", "gwei")
})

receipt = web3.eth.wait_for_transaction_receipt(tx)
print("✅ Dummy data tx mined:", receipt.transactionHash.hex())

✅ Dummy data tx mined: 584bc5bf568a9df5e419668cf65269aebf302b988397da7d922ed254d6dbe499


5. Verify it went through

After mining, call again to see the updated count and fetch the index.

In [33]:
# Cell 5: verify storage
new_total = contract.functions.getTotalRecords().call()
print(f"🔢 New total entries: {new_total}")

first = contract.functions.getRecord(0).call()
print("📦 First stored record:", first)

🔢 New total entries: 1
📦 First stored record: [1748075473, 'TEST001', 'Temperature', '22.5°C']
